In [ ]:
# initial setup stuff here

!git clone https://github.com/Glowman554/gpt-2

import os

if not os.getcwd().__contains__("gpt-2"):
  GPT2_DIR = os.path.join(os.getcwd(), "gpt-2")
  os.chdir(GPT2_DIR)

!sudo pip3 install -r requirements.txt
!python3 model/download_pretrained.py 124M

In [ ]:
from pathlib import Path
from shutil import copyfile

def read_latest_model(path):
  with open(path + "/checkpoint") as f:
    checkpoint_data = f.read()

    curr_model = None

    for i in checkpoint_data.split("\n"):
      checkpoint_entry = i.split(": ")
      if checkpoint_entry[0].strip() == "model_checkpoint_path":
        curr_model = checkpoint_entry[1].strip().replace("\"", "")

    return curr_model

def copy_file_same_same(_from, _to, file):
  copyfile(_from + "/" + file, _to + "/" + file)  

def copy_latest_model(_from, _to):
   Path(_to).mkdir(parents=True, exist_ok=True)
   curr_model = read_latest_model(_from)

   copy_file_same_same(_from, _to, "counter")
   copy_file_same_same(_from, _to, f"{curr_model}.data-00000-of-00001")
   copy_file_same_same(_from, _to, f"{curr_model}.index")
   copy_file_same_same(_from, _to, f"{curr_model}.meta")

   with open(_to + "/checkpoint", "w") as f:
    f.write(f"model_checkpoint_path: \"{curr_model}\"")

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

model_name = input("Model to load >")

copy_latest_model("/gdrive/My Drive/" + model_name, "checkpoint/run1/")

drive.flush_and_unmount()

In [ ]:
from google.colab import files
uploaded = files.upload()

for i in uploaded:
  try:
    copyfile(i, "dataset.txt")
    os.remove(i)
  except:
    pass

In [ ]:
!python3 model/encoder.py dataset.txt dataset.npz

In [ ]:
!python3 model/train.py --dataset dataset.npz --save_every 1000 --sample_every 1000 --sample_num 1

In [ ]:
!python3 model/interact.py

In [ ]:
!python3 model/interact.py --unconditional

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

model_name = input("Model to save >")

copy_latest_model( "checkpoint/run1/", "/gdrive/My Drive/" + model_name)

drive.flush_and_unmount()